# importaciones 

In [1]:

import os, json, random, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

c:\Users\USER\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Funciones a usar

In [2]:
RNG_SEED = 42
random.seed(RNG_SEED)
np.random.seed(RNG_SEED)
os.makedirs("out", exist_ok=True)

def l2_normalize(X: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(X, axis=1, keepdims=True)
    n[n == 0] = 1.0
    return X / n

def cosine_sim(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    A_ = l2_normalize(A)
    B_ = l2_normalize(B)
    return A_ @ B_.T

def recall_at_k(ranked_indices, gt_indices, k=10):
    hits = 0
    for r, gt in zip(ranked_indices, gt_indices):
        hits += int(gt in r[:k])
    return hits / len(gt_indices)

def show_examples(df: pd.DataFrame, idxs, title="Ejemplos"):
    print(f"\n--- {title} ---")
    for i in idxs[:5]:
        print(f"[{i}] {df.iloc[i]['Texto']}  ->  {df.iloc[i]['Categoría']}")


# Carga de dataset 

In [3]:

df = pd.read_csv("data/nlp_prueba_cc0c2_large.csv")
df = df.dropna(subset=["Texto"]).reset_index(drop=True)
len(df), df.head()


(10005,
                                                Texto Categoría
 0       La tokenización es clave para procesar texto  Positivo
 1             No entiendo los embeddings vectoriales  Negativo
 2         Los LLMs son impresionantes pero complejos   Neutral
 3               El curso de NLP es fascinante y útil  Positivo
 4  La programación en Python es complicada al pri...  Negativo)

# Oraciones y Consultas

In [4]:

n_oraciones = min(8000, len(df) - 200)  
n_consultas = 200

oraciones  = df.iloc[:n_oraciones].reset_index(drop=True).copy()
consultas = df.iloc[n_oraciones:n_oraciones+n_consultas].reset_index(drop=True).copy()

print(len(oraciones), len(consultas))
show_examples(oraciones, [0, 100, 500, 1000, 1500], "oraciones")
show_examples(consultas, [0, 10, 20, 30, 40], "consultas")


texto2idx = {t: i for i, t in enumerate(oraciones["Texto"].tolist())}
gt = []
mask_has_gt = []
for q in consultas["Texto"].tolist():
    if q in texto2idx:
        gt.append(texto2idx[q])
        mask_has_gt.append(True)
    else:
        gt.append(-1)
        mask_has_gt.append(False)

valid_mask = np.array(mask_has_gt)
print(f"Consultas con ground-truth en oraciones: {valid_mask.sum()} / {len(consultas)}")

top_k = 10


8000 200

--- oraciones ---
[0] La tokenización es clave para procesar texto  ->  Positivo
[100] La perplejidad parece confuso para procesar texto.  ->  Negativo
[500] La lematización requiere fascinante para procesar texto.  ->  Positivo
[1000] No entiendo cómo funciona la lematización, es innovador.  ->  Positivo
[1500] Implementar regularización se usa para lento en proyectos reales.  ->  Negativo

--- consultas ---
[0] Los modelos de lenguaje son eficiente pero fascinante.  ->  Positivo
[10] Entender los transformers parece eficiente en el curso de NLP.  ->  Positivo
[20] No entiendo cómo funciona la lematización, es fundamental.  ->  Neutral
[30] Entender los lematización se usa para difícil en el curso de NLP.  ->  Negativo
[40] Entender los LLMs se usa para esencial en el curso de NLP.  ->  Positivo
Consultas con ground-truth en oraciones: 149 / 200


# TF-IDF RECUPERACION COSENO

In [5]:

tfidf = TfidfVectorizer(
    lowercase=True,
    strip_accents="unicode",
    ngram_range=(1, 2),
    min_df=2
)
Xc = tfidf.fit_transform(oraciones["Texto"].tolist())
Xq = tfidf.transform(consultas["Texto"].tolist())


Xc_n = normalize(Xc, norm="l2", copy=True)
Xq_n = normalize(Xq, norm="l2", copy=True)

scores_tfidf = Xq_n @ Xc_n.T       
rank_tfidf = np.argsort(-scores_tfidf.toarray(), axis=1)[:, :top_k]

recall10_tfidf = recall_at_k(rank_tfidf[valid_mask], np.array(gt)[valid_mask], k=10)
print(f"TF-IDF Recall@10: {recall10_tfidf:.3f}")


TF-IDF Recall@10: 0.993


# EMBEDING DE OREACIONES Y RECUPERACION 

In [6]:

MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"  
st_model = SentenceTransformer(MODEL_NAME)
print("Modelo SBERT cargado:", MODEL_NAME)


emb_corpus = st_model.encode(
    oraciones["Texto"].tolist(),
    batch_size=64, show_progress_bar=False, normalize_embeddings=True
)
emb_queries = st_model.encode(
    consultas["Texto"].tolist(),
    batch_size=64, show_progress_bar=False, normalize_embeddings=True
)
emb_corpus = np.asarray(emb_corpus)
emb_queries = np.asarray(emb_queries)
print("Emb shapes:", emb_corpus.shape, emb_queries.shape)

S = cosine_sim(emb_queries, emb_corpus)
rank_emb = np.argsort(-S, axis=1)[:, :top_k]

recall10_emb = recall_at_k(rank_emb[valid_mask], np.array(gt)[valid_mask], k=10)
print(f"Embeddings ({MODEL_NAME}) Recall@10: {recall10_emb:.3f}")


c:\Users\USER\anaconda3\envs\nlp\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Modelo SBERT cargado: paraphrase-multilingual-MiniLM-L12-v2
Emb shapes: (8000, 384) (200, 384)
Embeddings (paraphrase-multilingual-MiniLM-L12-v2) Recall@10: 0.987


# COMPARACION RAPIDA Y EJEMPLOS DE VECINOS 

In [7]:
print(f"\nResumen Recall@10 (exact-match, {valid_mask.sum()} consultas válidas):")
print(f"  TF-IDF   : {recall10_tfidf:.3f}")
print(f"  Embeddings ({emb_backend}): {recall10_emb:.3f}")

# Mostrar vecinos de 3 consultas
def topn_text(rank, n=3, k=5):
    for qi in n:
        q_text = consultas.iloc[qi]["Texto"]
        idxs = rank[qi][:k]
        print("\nQ:", q_text)
        for j, di in enumerate(idxs, 1):
            print(f"  {j:2d}) [{di}] {oraciones.iloc[di]['Texto']}")

topn_text(rank_tfidf, n=[0,1,2], k=5)
topn_text(rank_emb,   n=[0,1,2], k=5)



Resumen Recall@10 (exact-match, 149 consultas válidas):
  TF-IDF   : 0.993


NameError: name 'emb_backend' is not defined

# PCA 2D EJEMPLOS VECINOS 

In [ ]:
# PCA 2D sobre una muestra del corpus (para que sea legible)
sample_n = min(1500, emb_corpus.shape[0])
idx_s = np.random.choice(emb_corpus.shape[0], size=sample_n, replace=False)
Z = emb_corpus[idx_s]

pca = PCA(n_components=2, random_state=RNG_SEED)
Z2 = pca.fit_transform(Z)

plt.figure(figsize=(6,6))
plt.scatter(Z2[:,0], Z2[:,1], s=6, alpha=0.6)
plt.title(f"PCA 2D de embeddings SBERT ({MODEL_NAME})")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.tight_layout()
plt.savefig("out/pca_embeddings_2d.png", dpi=160)
plt.show()

print("Figura guardada en out/pca_embeddings_2d.png")


# Variante opcional de evaluación

In [ ]:
def recall_by_label_at_k(rank, query_labels, corpus_labels, k=10):
    hits = 0
    for qi, idxs in enumerate(rank):
        labs = corpus_labels.iloc[idxs[:k]].tolist()
        hits += int(query_labels.iloc[qi] in labs)
    return hits / len(query_labels)

recall10_tfidf_cat = recall_by_label_at_k(rank_tfidf, consultas["Categoría"], oraciones["Categoría"], k=10)
recall10_emb_cat   = recall_by_label_at_k(rank_emb,   consultas["Categoría"], oraciones["Categoría"], k=10)

print(f"TF-IDF  Recall@10 (por categoría): {recall10_tfidf_cat:.3f}")
print(f"SBERT   Recall@10 (por categoría): {recall10_emb_cat:.3f}")


# Guardar métricas y reproducibilidad

In [ ]:
results = {
    "seed": RNG_SEED,
    "n_corpus": int(len(oraciones)),
    "n_queries": int(len(consultas)),
    "valid_queries_exact_match": int(valid_mask.sum()),
    "recall10_tfidf_exact": float(recall10_tfidf),
    "recall10_emb_exact": float(recall10_emb),
    "recall10_tfidf_by_label": float(recall10_tfidf_cat),
    "recall10_emb_by_label": float(recall10_emb_cat),
    "model_name": MODEL_NAME,
}
with open("out/metrics.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Métricas guardadas en out/metrics.json")
